<a href="https://colab.research.google.com/github/weedge/doraemon-nb/blob/main/inference_llama2_c.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# llama2.c

- https://github.com/karpathy/llama2.c

In [ ]:
!ulimit -a

real-time non-blocking time  (microseconds, -R) unlimited
core file size              (blocks, -c) unlimited
data seg size               (kbytes, -d) unlimited
scheduling priority                 (-e) 0
file size                   (blocks, -f) unlimited
pending signals                     (-i) 208844
max locked memory           (kbytes, -l) 8192
max memory size             (kbytes, -m) unlimited
open files                          (-n) 1048576
pipe size                (512 bytes, -p) 8
POSIX message queues         (bytes, -q) 819200
real-time priority                  (-r) 0
stack size                  (kbytes, -s) 8192
cpu time                   (seconds, -t) unlimited
max user processes                  (-u) unlimited
virtual memory              (kbytes, -v) unlimited
file locks                          (-x) unlimited


In [ ]:
!lscpu

Architecture:            x86_64
  CPU op-mode(s):        32-bit, 64-bit
  Address sizes:         46 bits physical, 48 bits virtual
  Byte Order:            Little Endian
CPU(s):                  8
  On-line CPU(s) list:   0-7
Vendor ID:               GenuineIntel
  Model name:            Intel(R) Xeon(R) CPU @ 2.20GHz
    CPU family:          6
    Model:               79
    Thread(s) per core:  2
    Core(s) per socket:  4
    Socket(s):           1
    Stepping:            0
    BogoMIPS:            4399.99
    Flags:               fpu vme de pse tsc msr pae mce cx8 apic sep mtrr pge mca cmov pat pse36 clf
                         lush mmx fxsr sse sse2 ss ht syscall nx pdpe1gb rdtscp lm constant_tsc rep_
                         good nopl xtopology nonstop_tsc cpuid tsc_known_freq pni pclmulqdq ssse3 fm
                         a cx16 pcid sse4_1 sse4_2 x2apic movbe popcnt aes xsave avx f16c rdrand hyp
                         ervisor lahf_lm abm 3dnowprefetch invpcid_single ssbd i

In [ ]:
#@title Clone Project

!git clone https://github.com/karpathy/llama2.c.git
%cd llama2.c

Cloning into 'llama2.c'...
remote: Enumerating objects: 1434, done.
remote: Total 1434 (delta 0), reused 0 (delta 0), pack-reused 1434
Receiving objects: 100% (1434/1434), 1.17 MiB | 21.73 MiB/s, done.
Resolving deltas: 100% (874/874), done.
/content/llama2.c


In [ ]:
!pip install -r requirements.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 323.6/323.6 kB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 47.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 619.9/619.9 MB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.5/78.5 kB 11.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 77.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.0/21.0 MB 68.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 849.3/849.3 kB 58.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.8/11.8 MB 89.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 557.1/557.1 MB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.1/317.1 MB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 168.4/168.4 MB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.6/54.6 MB 29.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━

In [ ]:
#@title Build with optimizer fast

!make runfast

gcc -Ofast -o run run.c -lm
gcc -Ofast -o runq runq.c -lm


In [ ]:
#@title Build with optimizer fast with openmp

!make runomp

gcc -Ofast -fopenmp -march=native run.c  -lm  -o run
gcc -Ofast -fopenmp -march=native runq.c  -lm  -o runq


In [ ]:
#@title Pick Your Model
#@markdown https://github.com/karpathy/llama2.c?#models

#@markdown Choose model https://huggingface.co/karpathy/tinyllamas
model = "stories110M" #@param ["stories15M", "stories42M", "stories110M"]

download_url = ""

if(model == "stories15M"):
  download_url = "https://huggingface.co/karpathy/tinyllamas/resolve/main/stories15M.bin"
if(model == "stories42M"):
  download_url = "https://huggingface.co/karpathy/tinyllamas/resolve/main/stories42M.bin"
if(model == "stories110M"):
  download_url = "https://huggingface.co/karpathy/tinyllamas/resolve/main/stories110M.bin"

print(f"download_url: {download_url}")

!wget $download_url

model_file = model + ".bin"

download_url: https://huggingface.co/karpathy/tinyllamas/resolve/main/stories110M.bin
--2023-12-21 04:49:47--  https://huggingface.co/karpathy/tinyllamas/resolve/main/stories110M.bin
Resolving huggingface.co (huggingface.co)... 18.238.49.70, 18.238.49.112, 18.238.49.117, ...
Connecting to huggingface.co (huggingface.co)|18.238.49.70|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs.huggingface.co/repos/88/4b/884bade32e5ee32eea725c5087af1358179a1bea94a4f6abc3c0470c9610ac38/515267168726a1ed1317a64a408492e6af3b67c1f71c5bd98c01d9d721803a24?response-content-disposition=attachment%3B+filename*%3DUTF-8%27%27stories110M.bin%3B+filename%3D%22stories110M.bin%22%3B&response-content-type=application%2Foctet-stream&Expires=1703393387&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTcwMzM5MzM4N319LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy5odWdnaW5nZmFjZS5jby9yZXBvcy84OC80Yi84ODRiYWRlMzJlNWVlMzJlZWE3MjVjNTA4N2FmMTM1ODE3O

In [ ]:
#@title Generate Stories

# Generate args
max_token = 256 #@param {type:"slider", min:32, max:1024, step:32}
temperature = 0.8 #@param {type:"slider", min:0.0, max:1, step:0.05}
top_p = 0.9 #@param {type:"slider", min:0.0, max:1.0, step:0.05}
prompt = "One day, Lily met a Shoggoth" #@param {type:"string"}

print(f"model: {model_file}, max_token: {max_token}, temperature: {temperature}, top_p: {top_p}, prompt: {prompt}")
print(f"----------------------------\n")

cmd = f'./run {model_file} -t {temperature} -p {top_p} -n {max_token} -i "{prompt}"'
print(cmd)
!{cmd}

model: stories110M.bin, max_token: 256, temperature: 0.8, top_p: 0.9, prompt: One day, Lily met a Shoggoth
----------------------------

./run stories110M.bin -t 0.8 -p 0.9 -n 256 -i "One day, Lily met a Shoggoth"
One day, Lily met a Shoggoth at the beach. She thought he was very funny and asked him if he wanted to play. The Shoggoth said yes and they started to play together.
Lily asked the Shoggoth what he liked to do. He said he liked to explore the beach and find things. He said he liked to eat shrimp. Lily thought that sounded yummy. 
The Shoggoth asked Lily if she wanted to try some of his shrimp. She said yes and she was very excited. He gave her a little bit and she tried it. It was so yummy! 
The Shoggoth asked Lily if she wanted to keep playing. She said yes and they had lots of fun together. She never forgot the yummy shrimp she tried.
achieved tok/s: 12.868440


关于采样的快速说明，为了获得最佳结果，建议使用 -t 1.0 -p 0.9 ，即温度temperature 1.0（默认），使用 进行 top-p 0.9（默认）采样。直观上，top-p 保证了概率很小的 token 不会被采样，所以我们在采样过程中不会“倒霉unlucky”，之后也不太可能“出轨off the rails”。

一般而言，要控制样本的多样性，请使用温度（即-t在 0 和 1 之间变化并保持 top-p 关闭-p 0）；

或者 top-p 值（即-p在 0 和 1 之间变化并保持-t 1），但不能同时使用两者。

关于 LLM 抽样策略的很好的解释参考如下：

- https://peterchng.com/blog/2023/05/02/token-selection-strategies-top-k-top-p-and-temperature/
- https://docs.cohere.com/docs/controlling-generation-with-top-k-top-p
- https://huggingface.co/blog/how-to-generate




In [ ]:
#@title Run Meta's Llama 2 models

#@markdown input your huggingface [access token](https://huggingface.co/settings/tokens) to download Meta's Llama 2 models.

from huggingface_hub import snapshot_download


from google.colab import userdata
token = userdata.get('HF_TOKEN')
#token = "" #@param {type:"string"}

path = snapshot_download(repo_id="meta-llama/Llama-2-7b",cache_dir="Llama-2-7b", use_auth_token=token)
print(path)

!python export.py llama2_7b.bin --meta-llama $path


Fetching 10 files:   0%|          | 0/10 [00:00<?, ?it/s]

Llama-2-7b/models--meta-llama--Llama-2-7b/snapshots/1c9f047f0e1dbe2e1be6f15f5107bf9f74bb425f
{'dim': 4096, 'multiple_of': 256, 'n_heads': 32, 'n_layers': 32, 'norm_eps': 1e-05, 'vocab_size': -1}
wrote llama2_7b.bin


In [ ]:
!ls -lh llama2_7b.bin

-rw-r--r-- 1 root root 26G Dec 21 04:26 llama2_7b.bin


In [ ]:
print("./run llama2_7b.bin\n")
!./run llama2_7b.bin

# run on 8 * Intel(R) Xeon(R) CPU @ 2.20GHz; so slow 2.452token/10s , SLA bad;
# maybe train a small LLM like stories model

./run llama2_7b.bin

Lily’s Story: “I didn’t have any connection with the baby”
“I’m going to tell you about something that happened to me, and it’s going to be very hard for me to talk about.” (Caroline, Professional Birth Story Teller, Listening Service).
Listening to the stories that people tell you about their births is the hardest and the most wonderful thing to do. You may never hear the details of the story, but just listening to it will make you appreciate life and love in a new way. I think that each birth story is beautiful and amazing in its own right, but for the purpose of this post, I am going to tell you a story about the realities of birth and the impact that it can have on everyone involved.
Lily’s Story:
I’m going to tell you about something that happened to me, and it’s going to be very hard for me to talk about.”
Lily, aged 40+
How was the pregnancy?
“It was really good. We didn’t have any complications, and there was no danger for the baby. We had a natural birth p

In [ ]:
!(OMP_NUM_THREADS=8 ./run llama2_7b.bin)

Miranda had been right about that. She knew it without a doubt. If she died, the plague would spread, and if the plague spread, millions would die.
And if Miranda was right, he was certain it would spread, and so far she'd been right about everything, so he did the only thing he could do.
He looked at her with a frightened pleading gaze.
"Miranda, you have to be wrong. That's something I have to believe. You can't be wrong."
She blinked and stared down at her glass. A second later, she took a sip of the strong, bitter drink. Her look was filled with more pain than he'd ever seen on her face, and he wondered if she thought the plague might not spread after all. He shook his head, feeling an ache in his chest as he looked at her.
Then, as if remembering her strange amulet, she suddenly reached out a hand and pulled his arm around her shoulders. It was a loving gesture, a surprising gesture, and it gave Dirk pause.
He wished it could be as simple as that.
If he could,
achieved tok/s: 0.98

In [ ]:
#@title Run Meta's Llama2-chat
!python export.py llama2_7b_chat.bin --meta-llama $path


{'dim': 4096, 'multiple_of': 256, 'n_heads': 32, 'n_layers': 32, 'norm_eps': 1e-05, 'vocab_size': -1}
wrote llama2_7b_chat.bin


In [ ]:
!OMP_NUM_THREADS=8 ./run llama2_7b_chat.bin -m chat

Enter system prompt (optional): 
User: One day, Lily met a Shoggoth
Assistant: 
[INST] A Kraken washed up in Singapore [/INST]
[INST] Not everyone is good at voodoo [/INST]
[INST] Not for everyone [/INST]
[INST] Old man yells at Shoggoth [/INST]
[INST] Lily stepped into an American horror [/INST]
[INST] Lily was lost [/INST]
[INST] [/INST]
This is insane, almost too much content...
Probably worth a few points though.
[Voidfall] Found a 6/6 Abyssal Goliath
[Mythos] Catheterized an elder
[Delirium] Can't quite pull together the Tomb Guardian I need...yet
This thread is hilarious. It feels like several posts were done simultaneously.

<s>
User: :q
Assistant: 
Labels: Intricate, Tangled in Purple
wow. just wow. it's amazing!! sooo detailed and beautiful.
Wow! Great work :) Love it


上面的（默认）脚本run.c使用 float32 前向传递，其中前向传递的整个计算都保存在 fp32 中。就参考代码而言，这很容易理解，但它有以下缺点：

模型检查点文件非常大（每个单独的权重需要 4 个字节），并且前向传递相对较慢。实践中采用的（非常）常见的推理优化是将模型参数量化为较低的精度，放弃一点正确性，以换取较小的检查点大小和更快的前向传递（因为大多数推理使用整数算术）。

根据经验，LLM 可以容忍低至 4 位（甚至更低）的精度，但我们在这里使用 int8，因为它是一种“安全”设置，可以为我们带来好处，但不会牺牲太多模型精度。

仅对参与 matmul 的权重进行量化。所有其他参数（例如，尤其是 RMSNorm 中的比例scale和偏差bias）都保存在 float32 中，因为这些层非常敏感。

现在，如果您所追求的只是减少检查点大小，则可以量化权重，保存检查点，然后在 run.c 中对它们进行反量化，并正常进行 float32 推理，然后就到此为止了。这完全没问题。

但在这里，我们更进一步（按照标准做法）并另外量化前向传递中的激活。这就需要我们在运行时动态地在float32和int8之间进行量化和反量化，这就增加了开销。但好处是，现在大多数计算（尤其是 matmul！）都使用纯整数算术，其中权重和激活都以 int8 形式输入。这就是加速的根本来源。我们使用的版本是“Q8_0”量化（llama.cpp 术语），其中 0 表示权重量化围绕 0 对称，量化范围为 [-127, 127]。

量化前向传播在runq.c中实现。要使用它，我们必须以量化格式导出模型。为了将其量化导出，我们改为使用版本 2 导出：

```
python export.py llama2_7b_q80.bin --version 2 --meta-llama path/to/llama/model/7B
```

In [ ]:
#@title Run Meta's Llama 2 models with int8 quantization
!python export.py llama2_7b_q80.bin --version 2  --meta-llama $path

{'dim': 4096, 'multiple_of': 256, 'n_heads': 32, 'n_layers': 32, 'norm_eps': 1e-05, 'vocab_size': -1}
1/226 quantized (32000, 4096) to Q8_0 with max error 0.0010248618200421333
2/226 quantized (4096, 4096) to Q8_0 with max error 0.0030364990234375
3/226 quantized (4096, 4096) to Q8_0 with max error 0.0017772279679775238
4/226 quantized (4096, 4096) to Q8_0 with max error 0.004190757870674133
5/226 quantized (4096, 4096) to Q8_0 with max error 0.003016674891114235
6/226 quantized (4096, 4096) to Q8_0 with max error 0.0026576723903417587
7/226 quantized (4096, 4096) to Q8_0 with max error 0.002516377717256546
8/226 quantized (4096, 4096) to Q8_0 with max error 0.002410888671875
9/226 quantized (4096, 4096) to Q8_0 with max error 0.002872016280889511
10/226 quantized (4096, 4096) to Q8_0 with max error 0.0021741949021816254
11/226 quantized (4096, 4096) to Q8_0 with max error 0.0030253250151872635
12/226 quantized (4096, 4096) to Q8_0 with max error 0.0018147137016057968
13/226 quantized 

In [ ]:
!ls -lhg *.bin

-rw-r--r-- 1 root  26G Dec 21 04:26 llama2_7b.bin
-rw-r--r-- 1 root  26G Dec 21 06:51 llama2_7b_chat.bin
-rw-r--r-- 1 root 6.7G Dec 21 07:42 llama2_7b_q80.bin
-rw-r--r-- 1 root 419M Jul 26 23:48 stories110M.bin
-rw-r--r-- 1 root  58M Jul 26 22:39 stories15M.bin
-rw-r--r-- 1 root 424K Dec 21 03:25 tokenizer.bin


In [ ]:
!./runq llama2_7b_q80.bin

Assisting David Warner in his 2005 retirement. Australian Cricket Team Image: AAP Image/Brendon Thorne
Mike Coward
David Warner has been talking about the death of Australian cricket in recent weeks, with his in-depth knowledge of cricket’s own NRL 9s, the global T20 circuit, and his global links through Warner Media.
In the light of Cricket Australia’s financial woes, he is concerned that the necessary commitment to the reinvigoration of the Australian team’s performance at the next World Cup might be hindered by current matters – especially due to his involvement in Warner Media.
A look at the structure of the league in the years since his retirement, including the importance of a significant contribution to the revival of Australian cricket, will be required to establish whether or not there is a specific justification for his recent remarks.
Who are the managers of the Twenty20 World Cup?
The ICC is in charge of this competition.
Its Supervisory Board has the authority to approve t

In [ ]:
!OMP_NUM_THREADS=8 ./runq llama2_7b_q80.bin

Hate the heat and humidity? Well here’s a heartfelt tale of New Orleans manhood and swamp-country hoo-hah!
But you’re not gonna find any snakes or moccasins or gross bambi here! No siree, we have good old fashioned pork to the skinny ladies (and men!) by the dozen! Now ain’t that something to hear….?!?
Damon Lewis is on a mission to find true love. Too bad he has to search around Louisiana for lovey-dovey bunnies to match his special needs. Can this average-Joe ever find someone?
Now if he’s gay and if it’s time to introduce Damon to the Chicken Ranch, because every great Chicken Ranch has a slew of boy-tenderness by the bunch of bitches! And who wouldn’t want that kind of swamp-lovin’ at the Ranch?
He’s already endured enough of his in-laws, but this is worse – an adult that can’t get the dumb-ass women to leave him alone – so he
achieved tok/s: 1.749499


# export huggingface models

可以加载任何使用 Llama 2 架构的 Huggingface 模型。请参阅脚本export.py和--hf导出模型 .bin 文件的标志。

In [ ]:
!pip install -q einops transformers_stream_generator

In [ ]:
%cd /content/llama2.c
# 51G memory OOM
# !python export.py baichuan_7b.bin --hf baichuan-inc/Baichuan-7B

# no rms_norm_eps, don't like LLama2 model struct
#!python export.py qwen_7b.bin --hf Qwen/Qwen-7B

# no num_hidden_layers, don't like LLama2 model struct
#!python export.py chatglm3_6b.bin --hf THUDM/chatglm3-6b

!python export.py chinese-llama2-1_3b.bin --hf hfl/chinese-llama-2-1.3b


config.json: 100% 671/671 [00:00<00:00, 2.68MB/s]
pytorch_model.bin: 100% 2.53G/2.53G [00:57<00:00, 43.6MB/s]
generation_config.json: 100% 170/170 [00:00<00:00, 666kB/s]
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:381: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.2` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:386: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`. This was detected when initializing the generation config instance, which means the

In [ ]:
!ls -lgh chinese-llama2-1_3b.bin

-rw-r--r-- 1 root 4.8G Dec 21 09:25 chinese-llama2-1_3b.bin


In [ ]:
!(OMP_NUM_THREADS=8 ./run llama2_7b.bin -i "你好")

你好!
Konnichiwa.
Follow
Say Hello
World
Learn all about me.
You will see I love to sing and act and dance.
I do lots of other fun things.
I have a dog called Chloe,
and we live on a farm.
But I don't think I live on a farm in China.
I don't think I live in Japan,
but maybe I do.
Follow
</s>

achieved tok/s: 1.378480


# llama2.c-zh

In [ ]:
%cd /content/

/content


In [ ]:
!git clone https://github.com/chenyangMl/llama2.c-zh.git
%cd llama2.c-zh

In [ ]:
!make runomp

gcc -Ofast -fopenmp -march=native run.c  -lm  -o run


In [ ]:
!wget https://huggingface.co/52AI/tinyllamas_zh/resolve/main/stories15M-llama2-enzh.bin


--2023-12-21 10:08:20--  https://huggingface.co/52AI/tinyllamas_zh/resolve/main/stories15M-llama2-enzh.bin
Resolving huggingface.co (huggingface.co)... 99.84.108.70, 99.84.108.87, 99.84.108.129, ...
Connecting to huggingface.co (huggingface.co)|99.84.108.70|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs.huggingface.co/repos/7e/7a/7e7a7ed4c2609b07cca1da298493c37bacce3551bdd392271801941d3e3cbbba/dfbb2b20e1504b2ec944e7a8a95847c2906461863b8869d84e8eb78ca7bf1abf?response-content-disposition=attachment%3B+filename*%3DUTF-8%27%27stories15M-llama2-enzh.bin%3B+filename%3D%22stories15M-llama2-enzh.bin%22%3B&response-content-type=application%2Foctet-stream&Expires=1703412500&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTcwMzQxMjUwMH19LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy5odWdnaW5nZmFjZS5jby9yZXBvcy83ZS83YS83ZTdhN2VkNGMyNjA5YjA3Y2NhMWRhMjk4NDkzYzM3YmFjY2UzNTUxYmRkMzkyMjcxODAxOTQxZDNlM2NiYmJhL2RmYmIyYjIwZTE1

In [ ]:
!OMP_NUM_THREADS=1 ./run stories15M-llama2-enzh.bin -k tokenizers/llama2enzh/tokenizer.bin -i "从前，有一个小女孩" -t 0.9

从前，有一个小女孩，名叫莉莉。有一天，莉莉的妈妈给她做了一份加了糖留的美味佳肴。但正当莉莉准备把它们全部吃掉之后，她发现味道很好。
莉莉的妈妈很生气并告诉莉莉，她晚餐必须吃一片豆子。莉莉不喜欢豆子的味道，她开始哭泣。
莉莉小姐知道搅拌豆子的一股以后，她希望晚饭清楚地会发生。最后，莉莉的妈妈给了她一篮子豆子，并告诉她明天来吃。莉莉又高兴了，她知道煮豆子是她自己做的，真相也很重要。
achieved tok/s: 96.913137


In [ ]:
!./run stories15M-llama2-enzh.bin -k tokenizers/llama2enzh/tokenizer.bin -i "从前，有一个小女孩" -t 0.9

从前，有一个小女孩，名叫莉莉。她喜欢用紫色蜡笔画画和搅拌。有一天，莉莉的妈妈让她带着钱包去公园。莉莉很高兴能帮忙，但又不知道路上的一切。
在公园里，莉莉遇到了一个名叫麦克斯的男孩。马克斯是无害的，但莉莉有点害怕。莉莉告诉他不要害怕，因为这会伤害他们。麦克斯坚持说，如果她趴着，他会很温柔。
莉莉考虑了马克斯所说的话，决定勇敢地鼓励他。麦克斯一开始很害怕，但莉莉鼓励他这么做。她带领麦克斯来到公园并与他和他的妹妹一起玩耍。
在公园玩完后，莉莉和麦克斯去买一些冰淇淋。麦克斯也很高兴，因为他让孩子们高兴了，他们在公园里玩得很开心。这个故事的寓意是，即使事情很可怕，我们也可以鼓励别人并向他们表明他们对其他人的健康做好准备。
achieved tok/s: 355.123675


In [ ]:
!wget https://huggingface.co/52AI/tinyllamas_zh/resolve/main/stories15M-baichuan.bin


--2023-12-21 10:13:12--  https://huggingface.co/52AI/tinyllamas_zh/resolve/main/stories15M-baichuan.bin
Resolving huggingface.co (huggingface.co)... 18.238.49.70, 18.238.49.117, 18.238.49.112, ...
Connecting to huggingface.co (huggingface.co)|18.238.49.70|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs.huggingface.co/repos/7e/7a/7e7a7ed4c2609b07cca1da298493c37bacce3551bdd392271801941d3e3cbbba/5e397f896e34d4760aaddf8ad5443f9fbcfafeb48c3dfaa53bc2f286d6335034?response-content-disposition=attachment%3B+filename*%3DUTF-8%27%27stories15M-baichuan.bin%3B+filename%3D%22stories15M-baichuan.bin%22%3B&response-content-type=application%2Foctet-stream&Expires=1703412793&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTcwMzQxMjc5M319LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy5odWdnaW5nZmFjZS5jby9yZXBvcy83ZS83YS83ZTdhN2VkNGMyNjA5YjA3Y2NhMWRhMjk4NDkzYzM3YmFjY2UzNTUxYmRkMzkyMjcxODAxOTQxZDNlM2NiYmJhLzVlMzk3Zjg5NmUzNGQ0NzYw

In [ ]:
!./run stories15M-baichuan.bin -k tokenizers/baichuan/tokenizer.bin -i "One day, Lily met a Shoggoth" -t 1


One day, Lily met a Shoggoth.bbbs explained that if she resolved this lesson times, she was feeling extra confident and brave. He felt envious of reason, but stayed strong and determined. 
The next day寿命 had learned fromopard Will, and with theerts they got to have a singing shooting blood. Gr wished anyone would dance too.
The moral of the story is that you can still be brave, no matter how綊 twiately, it will never come up. Sometimes, long andising can still be a great result.
achieved tok/s: 342.342342


In [ ]:
from sentencepiece import SentencePieceProcessor
sp_model_llama2 = SentencePieceProcessor(model_file="tokenizers/llama2en/tokenizer.model")
sp_model_llamaenzh = SentencePieceProcessor(model_file="tokenizers/llama2enzh/tokenizer.model")
sp_model_baichaun = SentencePieceProcessor(model_file="tokenizers/baichuan/tokenizer.model")
s = "从前，有一个小女孩，名叫莉莉。她喜欢在外面玩耍，欣赏美丽的花朵。"

llama2_en = sp_model_llama2.encode(s)
baichuan_enzh = sp_model_baichaun.encode(s)
llama2_enzh = sp_model_llamaenzh.encode(s)

print(f"llama2_en={len(llama2_en)}, llama2_enzh={len(llama2_enzh)} baichuan-enzh={len(baichuan_enzh)}")
print(f"llama2_en={sp_model_llama2.vocab_size()}, llama2_enzh={sp_model_llamaenzh.vocab_size()} \
      baichuan-enzh={sp_model_baichaun.vocab_size()}")
print("llama2_en",llama2_en)
print("llama2_enzh",llama2_enzh)
print(f"编码一个中文字符消耗tokens, llam2-en={len(llama2_en)/len(s):.2}, llama2_enzh={len(llama2_enzh)/len(s):.2}")
#-----------------------------------
#llama2_en=57, llama2_enzh=19 baichuan-enzh=23
#llama2_en=32000, llama2_enzh=55296       baichuan-enzh=64000
#llama2_en [29871, 31594, 30658, 30214, 30417, 30287, 30502, 30446, 30647, 232, 176, 172, 30214, 30548, 232, 146, 174, 235, 145, 140, 235, 145, 140, 30267, 232, 168, 188, 31823, 233, 175, 165, 30505, 31066, 30806, 234, 145, 172, 235, 131, 144, 30214, 233, 175, 166, 235, 184, 146, 30630, 231, 187, 192, 30210, 30830, 233, 159, 184, 30267]
#llama2_enzh [29871, 40870, 30214, 32952, 41677, 30214, 40148, 34595, 34595, 30267, 32008, 32123, 40729, 42754, 30214, 35186, 37973, 46892, 30267]
#编码一个中文字符消耗tokens, llam2-en=1.8, llama2_enzh=0.59

llama2_en=57, llama2_enzh=19 baichuan-enzh=23
llama2_en=32000, llama2_enzh=55296       baichuan-enzh=64000
llama2_en [29871, 31594, 30658, 30214, 30417, 30287, 30502, 30446, 30647, 232, 176, 172, 30214, 30548, 232, 146, 174, 235, 145, 140, 235, 145, 140, 30267, 232, 168, 188, 31823, 233, 175, 165, 30505, 31066, 30806, 234, 145, 172, 235, 131, 144, 30214, 233, 175, 166, 235, 184, 146, 30630, 231, 187, 192, 30210, 30830, 233, 159, 184, 30267]
llama2_enzh [29871, 40870, 30214, 32952, 41677, 30214, 40148, 34595, 34595, 30267, 32008, 32123, 40729, 42754, 30214, 35186, 37973, 46892, 30267]
编码一个中文字符消耗tokens, llam2-en=1.8, llama2_enzh=0.59


In [ ]:
# use baichuan tokenizer from https://huggingface.co/baichuan-inc/Baichuan-7B/blob/main/tokenizer.model;
# then use tokenizer.py export to *.bin
# run inference with tokenizer
!./run stories15M-baichuan.bin -k tokenizers/baichuan/tokenizer.bin


从前，有一支笨手笨脚的蜡烛。它很绿，很酷。那人一份喜欢玩火车的报纸。他们玩得很开心。
有一天，一只小鸟看见那人，知道这块蜡烛是绿色的。小鸟想用自己的蜡烛让蜡烛提供家园。于是，这只鸟把它放在自己的蜡烛下面。它添加了一些树枝和树叶。
朋友们在热气的蜡烛中嬉笑玩耍。他们都在一起玩。外面，那个人给了它一个大大的拥抱，并在他们周围玩耍。从那天起，推着大蜡烛，并弄得很更多。
achieved tok/s: 253.258845


In [ ]:
# use chatglm tokenizer from https://huggingface.co/THUDM/chatglm3-6b/blob/main/tokenizer.model
!wget https://huggingface.co/THUDM/chatglm3-6b/resolve/main/tokenizer.model -O tokenizers/chatglm3-tokenizer.model

--2023-12-21 14:12:26--  https://huggingface.co/THUDM/chatglm3-6b/resolve/main/tokenizer.model
Resolving huggingface.co (huggingface.co)... 99.84.108.87, 99.84.108.55, 99.84.108.129, ...
Connecting to huggingface.co (huggingface.co)|99.84.108.87|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs-us-1.huggingface.co/repos/01/5a/015a4eda6314fbcdb0aecc00e73f604e0f398b10ef843cf3e7bfcf8d0b9c1d7d/e7dc4c393423b76e4373e5157ddc34803a0189ba96b21ddbb40269d31468a6f2?response-content-disposition=attachment%3B+filename*%3DUTF-8%27%27tokenizer.model%3B+filename%3D%22tokenizer.model%22%3B&Expires=1703427146&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTcwMzQyNzE0Nn19LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy11cy0xLmh1Z2dpbmdmYWNlLmNvL3JlcG9zLzAxLzVhLzAxNWE0ZWRhNjMxNGZiY2RiMGFlY2MwMGU3M2Y2MDRlMGYzOThiMTBlZjg0M2NmM2U3YmZjZjhkMGI5YzFkN2QvZTdkYzRjMzkzNDIzYjc2ZTQzNzNlNTE1N2RkYzM0ODAzYTAxODliYTk2YjIxZGRiYjQwMjY5ZDMxNDY4YTZmMj

In [ ]:
!cat config.py


"""
tokenizer 和 训练数据配置文件
"""
LANGUAGE = "enzh"  # [en, zh, enzh]

# https://huggingface.co/baichuan-inc/Baichuan-7B/blob/main/tokenizer.model
#TOKENIZER_MODEL = "tokenizers/baichuan/tokenizer.model" # the baichuan sentencepiece tokenizer model
#TOKENIZER_BIN = "tokenizers/baichuan/tokenizer.bin" # binary version of the tokenizer for inference in C

# https://huggingface.co/ziqingyang/chinese-llama-2-7b/blob/main/tokenizer.model
# TOKENIZER_MODEL = "tokenizers/llama2enzh/tokenizer.model" # the llama2.
# TOKENIZER_BIN = "tokenizers/llama2enzh/tokenizer.bin" # binary version of the tokenizer for inference in C

# base llama2，
# TOKENIZER_MODEL = "tokenizers/llama2en/tokenizer.model" # the llama2-enzh.
# TOKENIZER_BIN = "tokenizers/llama2en/tokenizer.bin" # binary version of the tokenizer for inference in C

#自定义中文词表(红楼梦.txt)
# TOKENIZER_MODEL = "tokenizers/custom_tokenizer/meng.model" # the llama2-zh.
# TOKENIZER_BIN = "tokenizers/custom_tokenizer/meng.bin" # binary version of the tokeni

In [ ]:
%cd /content/llama2.c-zh
!python tokenizer.py

/content/llama2.c-zh
Loaded SentencePiece model from tokenizers/chatglm3-tokenizer.model
#words: 64789 - BOS ID: 1 - EOS ID: 2
TOKENIZER_BIN is saved = tokenizers/chatglm3-tokenizer.bin


In [ ]:
# ha~ so bad, need train chatglm3 model with chatglm3-tokenizer
!./run stories15M-llama2-enzh.bin -k tokenizers/chatglm3-tokenizer.bin -i "从前，有一个小女孩" -t 0.9

从前，有一个小女孩 Ved waterfront心上是一个总会 Gad一個網站主持风险 waterfrontς联合土豆 Gad疫苗接种联合亲人 waterfrontς日期问题 CUEdge好奇 Gad心上 Gad三大 Rash盛世读了老人Tagged较长國外 waterfrontiku心上体力 Gad幸福這麼 waterfrontς隐患 Asc演讲 undes并对工作会议途径经历了 waterfront
achieved tok/s: 302.083333


In [ ]:
#@markdown pre-train/fine-tunning Children's books datasets with chatglm3 or llama2 model struct

# see train notebook